In [1]:
import numpy as np
import cv2
from PIL import Image


In [63]:
import numpy as np
import cv2

class Contour:
    def snake(img, x, y, alpha=0.001, beta=0.4, gamma=100, sigma=20, iterations=10000):
        """
        The snake algorithm to segment image
        Parameters
        ----------
        img : ndarray
            input image
        x, y : ndarray
            X-coordinate and Y-coordinate of the initial contour
        alpha, beta: number
            The set of parameters of internal energy
        gamma : number
            Parameter controlling the external energy
        sigma : number
            Standard deviation
        iterations : number
            The number of iterations
        """

        # compute the matrix
        N = np.size(x)
        a = gamma * (2 * alpha + 6 * beta) + 1
        b = gamma * (-alpha - 4 * beta)
        c = gamma * beta
        p = np.zeros((N, N))
        p[0] = np.c_[a, b, c, np.zeros((1, N - 5)), c, b]
        for i in range(N):
            p[i] = np.roll(p[0], i)
        p = np.linalg.inv(p)

        # filter the image
        smoothed = cv2.GaussianBlur((img - img.min()) / (img.max() - img.min()), (89, 89), sigma)
        giy, gix = np.gradient(smoothed)
        gmi = (gix ** 2 + giy ** 2) ** 0.5
        gmi = (gmi - gmi.min()) / (gmi.max() - gmi.min())
        Iy, Ix = np.gradient(gmi)

        # avoid the curvature evolve to the outside of the image
        def fmax(x, y):
            x[x < 0] = 0
            y[y < 0] = 0
            x[x > img.shape[1] - 1] = img.shape[1] - 1
            y[y > img.shape[0] - 1] = img.shape[0] - 1
            return y.round().astype(int), x.round().astype(int)

        for i in range(iterations):
            fex = Ix[fmax(x, y)]
            fey = Iy[fmax(x, y)]
            x = np.dot(p, x + gamma * fex)
            y = np.dot(p, y + gamma * fey)
        return x, y

    def apply_snake(img, x, y, width, height, alpha, beta, gamma):
        # Load the image and define the initial contour points
        img = cv2.imread(img, 0)
        draw_image = img
        t = np.linspace(0, 2 * np.pi, 60, endpoint=True)
        x_0 = x + width / 2 + width / 2 * np.sin(t)
        y_0 = y + height / 2 + height / 2 * np.cos(t)
        img = cv2.Canny(img, 50, 150, apertureSize=3)

        # Run the snake algorithm using a greedy algorithm
        updated_contour = Contour.snake(img, x_0, y_0, alpha=alpha, beta=beta, gamma=gamma)
        # convert updated_contour to array of tuple
        updated_contour = np.array([updated_contour[0], updated_contour[1]]).T
        # update contour to array of int
        updated_contour = updated_contour.astype(int)

        # Display the result
        result = cv2.cvtColor(draw_image, cv2.COLOR_GRAY2RGB)
        for i in range(len(updated_contour)):
            cv2.circle(result, (updated_contour[i][0], updated_contour[i][1]), radius=1, color=(50, 100, 200), thickness=2)
            cv2.circle(result, (int(x_0[i]), int(y_0[i])), radius=1, color=(50, 200, 100), thickness=2)

        # write the output image
        cv2.imwrite('ContourOutput.jpg', result)


In [68]:
image_path = 'cent-400247_640.jpg'  # Provide the path to your image
# Define initial contour parameters
x = 50  # Initial x-coordinate
y = 50  # Initial y-coordinate
width = 500  # Width of the contour
height = 500  # Height of the contour
alpha = 0.0001  # Alpha parameter
beta = 0.4  # Beta parameter
gamma = 100  # Gamma parameter

# Apply snake algorithm to detect contours
Contour.apply_snake(image_path, x, y, width, height, alpha, beta, gamma)